# Import Libraries

In [1]:
import os
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

In [2]:
os.chdir('/Users/sabbirdelowar/projects/climate-hazard')

# Define Paths

In [3]:
rainfall_tif = "data/interim/mean_annual_rainfall.tif"
districts_shp = "data/aoi/aoi-multipoly.shp"
output_dir = "results/individual-hazards"
os.makedirs(output_dir, exist_ok=True)
output_csv = os.path.join(output_dir, "rainfall_hazard.csv")

# Perform Zonal Statistics

In [4]:
districts = gpd.read_file(districts_shp)
districts = districts.to_crs("EPSG:4326")
pstats = zonal_stats(
    vectors=districts,
    raster=rainfall_tif,
    stats=['mean'],
    geojson_out=True
)

/Users/sabbirdelowar/anaconda3/lib/python3.11/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


# Build DataFrame

In [9]:
district_names = [feature['properties']['name'] for feature in pstats]
mean_rainfall = [feature['properties']['mean'] for feature in pstats]

rainfall_df = pd.DataFrame({
    "district": district_names,
    "mean_annual_rainfall_mm": mean_rainfall
})

# Scale to 0–1

In [12]:
min_val = rainfall_df['mean_annual_rainfall_mm'].min()
max_val = rainfall_df['mean_annual_rainfall_mm'].max()

rainfall_df['rainfall_hazard_score'] = (
    (rainfall_df['mean_annual_rainfall_mm'] - min_val) / (max_val - min_val)
)

    district  mean_annual_rainfall_mm  rainfall_hazard_score
0   Bagerhat                62.471035               0.167894
1  Bandarban                63.946386               0.187637
2    Barguna                64.844917               0.199661
3    Barisal                59.618431               0.129721
4      Bhola                69.963664               0.268159
